In [ ]:
import sys
#!{sys.executable} -m pip install

In [97]:
from numpy import vectorize
from quantities import UnitQuantity, dimensionless, C, N, m, A, mm, V, F, \
  pF, H, ohm, Hz, MHz, S, s, kg, mS, rad
from numpy import pi, log10, e, real, sqrt, imag, exp, log, arcsin, sin, cos, arccos, arctan
import numpy as np
from quantities import UnitQuantity, m, s, us, GHz, Hz, C, F, pF, N, A, Z_0
from pandas import DataFrame
from sympy import latex, solve, symbols
from IPython.display import display, Math
import re
from plotly.express import scatter, line


def Units(x, units):
    x.units = units
    return x

light_speed = 299792458 * m/s

c = UnitQuantity('c', light_speed)

epsilon0 = 8.854187817 * 10**(-12) * C**2/(N*m**2)
mu0 = 4*pi*10**(-7) * N/(A**2)

Np = UnitQuantity('Np', 1*dimensionless) #neper is a dimensionless unit that functions like the decible but with base e instead of base 10
dB = UnitQuantity('dB', dimensionless*Np/log10(e))
#dB = UnitQuantity('dB', 1)

nH = UnitQuantity('nH', H*1e-9)

j = 1j

Z_0

UnitQuantity('characteristic_impedance_of_vacuum', 1.0 * c*mu_0, 'Z_0')

# 1

In [61]:
beta = ((2*pi*5.2*GHz*sqrt(11.2)/light_speed)*(1+0.0022**2/8)).simplified

display(f'beta = {beta}')

'beta = 364.73024901056726 1/m'

# 3

In [74]:
alpha = Units(((pi*5.2*GHz*sqrt(11.2)/light_speed)*(0.0022)), Np/m)
display(f'alpha = {alpha}')

'alpha = 0.4012030311837901 Np/m'

# 5

In [126]:
alpha = Units(alpha, dB/m)

display(alpha)

dB_power_loss = 5.8*dB

power_decrease = 10**(-dB_power_loss/(10*dB))
power_decrease

display(f'Power decrease = {power_decrease}')

x = (1/alpha)*log(1/power_decrease)
x

array(0.17424026) * dB/m

'Power decrease = 0.2630267991895382 dimensionless'

array(7.66470008) * m/dB

# 6

In [128]:
def reflectionCoefficent(epsilon_r, mu_r, theta_incident, Type):
  c = 1/(mu0*epsilon0)**0.5
  refractive_index_air = 1.000/c #convert relative refractive index to absolute refractive index
  epsilon = epsilon0*epsilon_r
  mu = mu0*mu_r
  n2 = refractive_index_air
  n1 = (mu*epsilon)**0.5 #definition of refractive index
  theta_transmitted = arcsin(sin(theta_incident)*n1/n2) #Solving Snell's law for theta2

  #intrinsic impedance of free space
  eta2 = (mu0/epsilon0)**0.5
  #intrinsic impedance of medium
  eta1 = (mu/epsilon)**0.5

  if Type == 'p': #parallel polarization
    gamma = (eta1 * cos(theta_incident) - eta2 * cos(theta_transmitted)) \
      /(eta1 * cos(theta_incident) + eta2 * cos(theta_transmitted))
  elif Type == 's': #orthogonal polarization
    gamma = (eta2 * cos(theta_incident) - eta1 * cos(theta_transmitted))  \
    /(eta1 * cos(theta_transmitted) + eta2 * cos(theta_incident))
  else:
    print('Invalid value entered for type')
    return('error')

  return gamma

#note that, with an incidence angle of 0 degrees, the reflection coefficient is the same for both types of polarization
reflection_coefficient = reflectionCoefficent(epsilon_r = 2.33, mu_r = 1, theta_incident = 0, Type = 's')
#the reflection coefficient is the sqrt of reflectance

reflection_coefficient

array(0.2083703) * dimensionless

# 7 - 14

In [38]:
theta_incident = np.linspace(0, 90, num = 10000)

reflectance_p = abs(reflectionCoefficent(2.33, 1, theta_incident * pi / 180, Type = 'p')) #convert degrees to radians
reflectance_s = abs(reflectionCoefficent(2.33, 1, theta_incident * pi / 180, Type = 's'))

pReflectance_df = DataFrame({'Angle of Incidence (degrees)': theta_incident , 'P Reflectance': reflectance_p})
sReflectance_df = DataFrame({'Angle of Incidence (degrees)': theta_incident , 'S Reflectance': reflectance_s})
pReflectance_df

pReflectance_plot = scatter(pReflectance_df, x = 'Angle of Incidence (degrees)',  y = 'P Reflectance').update_layout(title={
        'text': "Reflectance vs Angle of Incidence for P Polarized Laser Beam",
        'y':0.95,
        'x':0.5,
        #'xanchor': 'center',
        'yanchor': 'top'})
pReflectance_plot.show()

sReflectance_plot = scatter(sReflectance_df, x = 'Angle of Incidence (degrees)',  y = 'S Reflectance').update_layout(title={
        'text': "Reflectance vs Angle of Incidence for S Polarized Laser Beam",
        'y':0.95,
        'x':0.5,
        #'xanchor': 'center',
        'yanchor': 'top'})
sReflectance_plot

C:\Users\mbles\AppData\Local\Temp\ipykernel_2960\2609042173.py:8: RuntimeWarning:

invalid value encountered in arcsin



# 15

$Z_C = \frac{1}{2\pi j f C}$

# 16

In [113]:
f = 3.5*GHz
cap = 1.2*pF

Zc = Units(-j/(2*pi*f*cap), ohm)

display(Math(latex(Zc)))

display(Math(f'\\text{{reactance }} = {round(imag(Zc), 4)}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# 17

$Z_{OC} = -j Z_0 \cot(\beta l)$

# 18

$Z_C = Z_{\text{char}} x \implies x = \frac{Z_c}{Z_{\text{char}}} = \frac{1}{2\pi j f C Z_{\text{char}}}$
<br></br>
$\Beta l = \arctan\left(\frac{1}{|x|}\right) = \arctan\left(2\pi f C Z_{\text{char}}\right)$

In [131]:
z_char = 68.2*ohm

display(f'electrical length = {arctan(((2*pi*cap*f*z_char)).simplified.magnitude)*180/pi}')

'electrical length = 60.942092934749'